**Chapter 6 --- Decision Trees (Revision Summary)**
=================================================

*Simple theory + formulas + minimal runnable snippets*

* * * * *



**1 --- What Is a Decision Tree?**
================================

A decision tree predicts by repeatedly **splitting the data** based on feature thresholds.

-   For classification → predicts a **class**

-   For regression → predicts a **numeric value**

A tree learns **rules** like:

`if feature A ≤ 2.7:
    if feature B > 1.3:
        class = 1
    else:
        class = 0
else:
    class = 2`

Trees split nodes to make child nodes **purer**.

No need for feature scaling.

* * * * *

**2 --- Decision Tree Classification**
====================================

**2.1 Impurity Measures**
-------------------------

A tree chooses splits that reduce impurity.

### **Gini Impurity**

Most common in sklearn:

`Gini = 1 - Σ (pᵢ²)`

Minimum = 0 (pure node).

### **Entropy (Information Gain)**

`Entropy = - Σ (pᵢ log₂ pᵢ)`

Information Gain = reduction in entropy.

Both produce similar trees.

* * * * *
**2.2 Classification Code Example**
-----------------------------------

`from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(
    criterion="gini",     # or "entropy"
    max_depth=4,
    random_state=0
)

model.fit(X_train, y_train)
pred = model.predict(X_test)`

* * * * *

**2.3 Important Classification Hyperparameters**
------------------------------------------------

| Hyperparameter | Meaning |
| --- | --- |
| **max_depth** | Maximum depth of the tree (main control for overfitting) |
| **min_samples_split** | Minimum samples required to split a node |
| **min_samples_leaf** | Minimum samples required in a leaf |
| **max_leaf_nodes** | Maximum leaves allowed |
| **criterion** | "gini" or "entropy" |
| **class_weight** | Handle class imbalance |

### **Overfitting control:**

Increase `min_samples_leaf`, reduce `max_depth`.

* * * * *

**3 --- Decision Tree Regression**
================================

Same idea, but predicts a continuous value.

* * * * *

**3.1 Regression Impurity Measures**
------------------------------------

Instead of Gini/Entropy, regression trees use **variance**.

### **MSE impurity (default)**

`impurity = mean( (y - mean(y))² )`

A tree selects splits that **reduce MSE**.

* * * * *

**3.2 Regression Code Example**
-------------------------------

`from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(
    max_depth=5,
    min_samples_leaf=4,
    random_state=0
)

reg.fit(X_train, y_train)
pred = reg.predict(X_test)`

* * * * *

**3.3 Regression Hyperparameters**
----------------------------------

Same as classification, except criterion options:

-   `criterion="squared_error"`

-   `criterion="friedman_mse"`

-   `criterion="absolute_error"`

* * * * *

**4 --- Handling Overfitting in Decision Trees**
==============================================

Trees **overfit VERY easily**.

Use:

### **Pruning / Pre-pruning**

-   `max_depth`

-   `min_samples_split`

-   `min_samples_leaf`

-   `max_leaf_nodes`

### **Post-Pruning**

(sklearn supports cost complexity pruning)

`ccp_alpha > 0     # bigger alpha = more pruning`

Example:

`tree_pruned = DecisionTreeClassifier(ccp_alpha=0.01)`

* * * * *

**5 --- Advantages & Limitations**
================================

### **Advantages**

-   Easy to interpret

-   No scaling needed

-   Handles nonlinear relations

-   Handles mixed feature types

-   Fast inference

### **Limitations**

-   High variance (unstable)

-   Overfits easily

-   Not as accurate as ensemble methods

Ensemble methods like **Random Forest**, **Extra Trees**, **Gradient Boosting** fix these issues.

* * * * *

**6 --- Visualizing Trees**
=========================

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))
plot_tree(model, filled=True, feature_names=feature_names)
plt.show()

**7 --- Feature Importance**
==========================

Each tree computes importance based on impurity reduction.


Higher = more important.

* * * * *

In [ ]:
model.feature_importances_

**8 --- Summary Table (One-Glance)**
==================================

| Type | Split Metric | Output | Notes |
| --- | --- | --- | --- |
| Classification Tree | Gini or Entropy | Class label | Overfits easily |
| Regression Tree | MSE | Numeric value | Same hyperparameters as classification |
| Pre-pruning | max_depth, min_samples_leaf | Controls overfitting | Most effective |
| Post-pruning | ccp_alpha | Removes weak splits | Very useful |
| Feature Importance | impurity reduction | ranking | Good interpretability |

* * * * *

**9 --- Full Minimal Example (Classification)**
=============================================

* * * * *

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(X_train, y_train)

print("Accuracy:", accuracy_score(y_test, clf.predict(X_test)))

**10 --- Full Minimal Example (Regression)**
==========================================

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

reg = DecisionTreeRegressor(max_depth=4)
reg.fit(X_train, y_train)

pred = reg.predict(X_test)
print("RMSE:", np.sqrt(mean_squared_error(y_test, pred)))

**How Do Decision Trees Split in Regression?**
==============================================

Unlike classification trees (which use Gini/Entropy),\
**Regression Trees use MSE (mean squared error) or variance** to choose splits.

Everything is based on one idea:

> **A good split creates child nodes where the target values are close to each other.**

* * * * *

 **Goal of Regression Trees**
===============================

For each candidate split:

`X_j ≤ threshold`

we compute how much it reduces the total error.

The attribute **and** the **threshold** giving the **largest error reduction** is selected.

Simple.

* * * * *

**Step 1 --- Define Impurity (Error) of a Node**
=================================================

For regression, impurity = variance (or MSE):

`Impurity(node) = (1/N) Σ (yᵢ - mean(y))²`

or simply the variance of values in that node.

* * * * *

**Step 2 --- Try Every Feature + Every Possible Split Point**
==============================================================

Given feature `X_j` with continuous values:

### How do we get candidate thresholds?

We take **sorted unique values** of that column and try **midpoints** between them.

Example:

`Values: [1, 3, 5]
Candidate thresholds: 2, 4`

So a split is:

`X_j ≤ threshold`

Tree tries ALL of them.

* * * * *

**Step 3 --- For Each Split, Compute Left and Right Node Errors**
==================================================================

Suppose a split creates:

-   Left node: N_L samples

-   Right node: N_R samples

Compute their impurities:

`I_L = impurity(left)
I_R = impurity(right)`

* * * * *

**Step 4 --- Compute Weighted Average Impurity After Split**
=============================================================

This is the **post-split impurity**:

`WeightedImpurity = (N_L / N_total) * I_L  +  (N_R / N_total) * I_R`

* * * * *

**Step 5 --- Compute Impurity Reduction (the "Gain")**
=======================================================

`Gain = Impurity_parent - WeightedImpurity`

The tree chooses the split with **maximum gain**.

* * * * *

 FULL FORMULA 
================

Let parent node impurity be:

`I_parent = (1/N) Σ (yᵢ - mean(y_parent))²`

Then best split = argmax over all (j, threshold):

`Gain(j, threshold) = I_parent - (N_L/N)*I_L - (N_R/N)*I_R`

That's it!

* * * * *

 **Intuition Summary**
========================

-   Try all features

-   Try all possible thresholds

-   Compute how much the split reduces variance

-   Choose the split that makes child nodes purer (less spread)

Continuous variables are handled naturally because split thresholds are just numeric points.

* * * * *

 Example (Small Numbers)
==========================

Suppose y-values in a node:

`[3, 4, 7, 8]`

Try split at <=4:

Left: [3,4]\
Right: [7,8]

Variance(left) = low\
Variance(right) = low

This is a good split.

Try split at <=7:

Left: [3,4,7]\
Right: [8]

Left variance = higher\
Right variance = zero\
But weighted average variance = worse

So <=4 is better.

* * * * *

 Sklearn Equivalent
=====================

Scikit-Learn uses exactly this, with criterion:

`criterion="squared_error"`


Final Understanding:
=======================

### **For regression:**

-   Impurity = MSE / variance

-   Split chosen based on maximum **variance reduction**

-   Continuous variables → try all possible numeric thresholds

-   Best split = mean reduction in spread of y-values


 Minimal Code: Print Best Split Manually
==========================================

In [ ]:
import numpy as np

y = np.array([3, 4, 7, 8])
X = np.array([2, 4, 6, 8])  # feature

def variance(x):
    return np.mean((x - np.mean(x))**2)

best_gain = -1
best_thresh = None

parent_imp = variance(y)

for t in np.unique(X):
    left = y[X <= t]
    right = y[X > t]
    if len(left)==0 or len(right)==0:
        continue
    wimp = (len(left)*variance(left) + len(right)*variance(right)) / len(y)
    gain = parent_imp - wimp
    if gain > best_gain:
        best_gain = gain
        best_thresh = t

best_thresh, best_gain